In [1]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import svm_builder
import importlib

In [2]:
feature_number = 105

X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)

(393345, 105)
(393345,)


下面这段是 Balance with Down sample，效果不好

In [5]:
import balance
importlib.reload(balance)

# down sample
balance_x, balance_y = balance.get_balance_class_downsample(X, y)
print(balance_x.shape)
print(balance_y.shape)

(7252, 105)
(7252,)


In [6]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(balance_x)
X_scaled = scaler.fit_transform(balance_x)

In [7]:
pca1 = PCA(n_components= 0.999).fit(X_scaled)
pca2 = PCA(n_components= 0.95).fit(X_scaled)
X_pca1 = pca1.transform(X_scaled)
X_pca2 = pca2.transform(X_scaled)
print(X_pca1.shape)
print(X_pca2.shape)

(7252, 80)
(7252, 42)


In [9]:
model_X = X_pca1
model_y = balance_y
print(model_X.shape, model_y.shape)

(7252, 80) (7252,)


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# devide X and y into train dataset and test dataset:
X_train, X_test, y_train, y_test = train_test_split(model_X, model_y, test_size=0.3, random_state=42)

In [12]:
import logistic_builder

In [13]:
log_model = logistic_builder.train_logistic_model(X_train, y_train)

Training set score of logistic: 0.747255840135097
Testing set score of logistic: 0.7078135259356533


In [14]:
y_pred = log_model.predict(X_test)
y_pred = y_pred.astype(int).tolist()
y_test = y_test.astype(int).tolist()

In [15]:
import scoring
importlib.reload(scoring)

<module 'scoring' from 'E:\\2023 Fall\\IML\\project\\iml-project\\scoring.py'>

In [16]:
scoring.score(y_pred, y_test)

Class 0: Accuracy: 0.9361213235294118, Precision: 0.8047138047138047, Recall: 0.746875
Class 1: Accuracy: 0.8708639705882353, Precision: 0.5587188612099644, Recall: 0.5
Class 2: Accuracy: 0.8736213235294118, Precision: 0.5693215339233039, Recall: 0.5993788819875776
Class 3: Accuracy: 0.9076286764705882, Precision: 0.6318840579710145, Recall: 0.7465753424657534
Class 4: Accuracy: 0.9471507352941176, Precision: 0.8235294117647058, Recall: 0.8209876543209876
Class 5: Accuracy: 0.9131433823529411, Precision: 0.7037037037037037, Recall: 0.6354515050167224
Class 6: Accuracy: 0.9604779411764706, Precision: 0.8411214953271028, Recall: 0.8852459016393442
mf1 score: 0.7048219228189334
acc_score: 0.7045036764705882
kappa_score: 0.6795205708658937


In [17]:
cnt = len(y_test)
acc = np.zeros(7)
for i in range(cnt):
    if y_test[i] == y_pred[i]:
        acc[int(y_test[i])] += 1
for i in range(7):
    print(f"Accuracy of class {i}: {acc[i]/y_test.count(i)}")

Accuracy of class 0: 0.746875
Accuracy of class 1: 0.5
Accuracy of class 2: 0.5993788819875776
Accuracy of class 3: 0.7465753424657534
Accuracy of class 4: 0.8209876543209876
Accuracy of class 5: 0.6354515050167224
Accuracy of class 6: 0.8852459016393442


(1968512, 105)
(1968512,)
